# Loading metabolomics data

In [7]:
%run plputil.py
metabolomics_list = [
    "WT_5uMv20nM_metabo.csv","yggS_v_WT_20nMPN_metabo.csv","yggS_v_WT_5uMPN_metabo.csv","pdxBJ_v_WT_20nMPN_metabo.csv"
]
metabolomics = {}
met_class = {}
for item in metabolomics_list:
    metabolomics[item[0:-11]] = {}
    met_class[item[0:-11]] = {}
    data = pd.read_csv("newdata/"+item,encoding='latin1')
    column_names = data.columns.tolist()
    for index,row in data.iterrows():
        metabolomics[item[0:-11]][row["BiGG Metabolite ID"]] = row[column_names[1]]
        if row["BiGG Metabolite ID"] not in met_class[item[0:-11]]:
            met_class[item[0:-11]][row["BiGG Metabolite ID"]] = {}
        if row[column_names[1]] > 0.3:
            met_class[item[0:-11]][row["BiGG Metabolite ID"]] = "U"
        elif row[column_names[1]] < -0.3:
            met_class[item[0:-11]][row["BiGG Metabolite ID"]] = "D"
        else:
            met_class[item[0:-11]][row["BiGG Metabolite ID"]] = "S"
util.save("metabolomics",metabolomics)
util.save("met_class",met_class)


Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/PLP/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1722452360.050656 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


WT_5uMv20nM_metabo.csv 0 0
yggS_v_WT_20nMPN_metabo.csv 0 0
yggS_v_WT_5uMPN_metabo.csv 0 0
pdxBJ_v_WT_20nMPN_metabo.csv 0 0


# Loading expression data

In [20]:
%run plputil.py
expression_list = [
    "WT_5uMv20nM_expression.csv","yggS_v_WT_20nMPN_expression.csv","yggS_v_WT_5uMPN_expression.csv","pdxBJ_v_WT_20nMPN_expression.csv"
]
expression = {}
for item in expression_list:
    expression[item[0:-15]] = {}
    data = pd.read_csv("newdata/"+item,encoding='latin1')
    column_names = data.columns.tolist()
    for index,row in data.iterrows():
        expression[item[0:-15]][row[column_names[0]]] = row[column_names[1]]
util.save("expression",expression)
raw_expression = {}
data = pd.read_csv("newdata/RawExpressionValues.csv",encoding='latin1')
column_names = data.columns.tolist()
for item in column_names:
    if item != "Blattner":
        raw_expression[item] = {}
        for index,row in data.iterrows():
            raw_expression[item][row["Blattner"]] = row[item]
util.save("raw_expression",raw_expression)

Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/PLP/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Running model with expression data

In [14]:
%run plputil.py
raw_expression = util.load("raw_expression")
fluxes = {}
growth_rates = {
    "WT_20nM_PN_AVE":0.3768,
    "WT_5uM_PN_AVE":0.3797,
    "yggS_20nM_PN_AVE":0.3768,
    "yggS_5uM_PN_AVE":0.3797,
    "pdxBJ_20nM_PN_AVE":0.2622,
    "pdxBJ_5uM_PN_AVE":0.4051
}
thresholds = {
    "WT_20nM_PN_AVE":4,
    "WT_5uM_PN_AVE":4,
    "yggS_20nM_PN_AVE":4,
    "yggS_5uM_PN_AVE":4,
    "pdxBJ_20nM_PN_AVE":4,
    "pdxBJ_5uM_PN_AVE":4
}
output = {}
for item in raw_expression.keys():
    output[item] = util.compute_expression_fluxes(util.model,thresholds[item],raw_expression[item],growth_rates[item],save_model=True)
util.save("threshold_results",output)

Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/PLP/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1722616232.803574 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722616244.777778 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722616258.8716311 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722616318.320249 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722616332.1386611 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722616346.008731 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722616407.57116 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


# Run optimized models

In [15]:
%run plputil.py
threshold_results = util.load("threshold_results")
fluxes = {}
for item in raw_expression.keys():
    model = cobra.io.json.from_json(threshold_results[item]["model"])
    solution = model.optimize()
    fluxes[item] = {}
    for fitem in solution.fluxes.keys():
        fluxes[item][fitem] = solution.fluxes[fitem]
util.save("fluxes",fluxes)

Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/PLP/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


ConnectionError: HTTPSConnectionPool(host='kbase.us', port=443): Max retries exceeded with url: /services/ws (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x68f80abb0>: Failed to establish a new connection: [Errno 61] Connection refused'))

1722619008.104019 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722619021.146337 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722619087.131304 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722619101.4112499 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722619164.3304582 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1722619177.151584 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


# Computing foldchanges

In [9]:
%run plputil.py
expression = util.load("expression")
raw_expression = util.load("raw_expression")
fluxes = util.load("fluxes")
reaction_expression = {}
for item in raw_expression.keys():
    reaction_expression[item] = {}
    for rxn in util.model.reactions:
        reaction_expression[item][rxn.id] = 0
        for gene in rxn.genes:
            if gene.id in raw_expression:
                reaction_expression[item][rxn.id] += raw_expression[gene.id][item]
pairings = {
    "yggS_20nM":["yggS_20nM_PN_AVE","WT_20nM_PN_AVE"],
    "yggS_5uM":["yggS_5uM_PN_AVE","WT_5uM_PN_AVE"],
    "pdxBJ_20nM":["pdxBJ_20nM_PN_AVE","WT_20nM_PN_AVE"],
    "pdxBJ_5uM":["pdxBJ_5uM_PN_AVE","WT_5uM_PN_AVE"]
}
util.save("pairings",pairings)
expression_fold_changes = {}
flux_fold_changes = {}
for item in pairings.keys():
    expression_fold_changes[item] = {}
    flux_fold_changes[item] = {}
    for rxn in util.model.reactions:
        if pairings[item][0] in reaction_expression and pairings[item][1] in reaction_expression:
            if reaction_expression[pairings[item][0]][rxn.id] == 0:
                if reaction_expression[pairings[item][1]][rxn.id] > 0:
                    expression_fold_changes[item][rxn.id] = -1
                else:
                    expression_fold_changes[item][rxn.id] = 0
            elif reaction_expression[pairings[item][1]][rxn.id] == 0:
                if reaction_expression[pairings[item][0]][rxn.id] > 0:
                    expression_fold_changes[item][rxn.id] = 1
                else:
                    expression_fold_changes[item][rxn.id] = 0
            else:
                expression_fold_changes[item][rxn.id] = math.log2(reaction_expression[pairings[item][0]][rxn.id]/reaction_expression[pairings[item][1]][rxn.id])
        if pairings[item][0] in fluxes and pairings[item][1] in fluxes:
            if abs(fluxes[pairings[item][0]][rxn.id]) < 0.0000001:
                if abs(fluxes[pairings[item][1]][rxn.id]) > 0.0000001:
                    flux_fold_changes[item][rxn.id] = -1
                else:
                    flux_fold_changes[item][rxn.id] = 0
            elif abs(fluxes[pairings[item][1]][rxn.id]) < 0.0000001:
                if abs(fluxes[pairings[item][0]][rxn.id]) > 0.0000001:
                    flux_fold_changes[item][rxn.id] = -1
                else:
                    flux_fold_changes[item][rxn.id] = 0
            else:
                flux_fold_changes[item][rxn.id] = math.log2(abs(fluxes[pairings[item][0]][rxn.id])/abs(fluxes[pairings[item][1]][rxn.id]))

util.save("expression_fold_changes",expression_fold_changes)
util.save("reaction_expression",reaction_expression)
util.save("flux_fold_changes",flux_fold_changes)

Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/PLP/nboutput when using KBDevUtils.output_dir


1722658776.235625 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Identifying cofactors in reactions

In [6]:
%run plputil.py
met_count = {}
for rxn in util.model.reactions:
    for met in rxn.metabolites:
        #Counting single instances of metabolites in reactions
        if met not in met_count:
            met_count[met] = 0
        met_count[met] += 1
data = []
for met in util.model.metabolites:
    data.append({"id":met.id,"name":met.name,"count":met_count[met]})
df = pd.DataFrame.from_records(data)
df.to_csv(util.output_dir+"/SingletonMetaboliteCount.csv")
cofactors = [
    "h_c",
    "h2o_c",
    "atp_c",
    "pi_c",
    "h_p",
    "adp_c",
    "h2o_p",
    "ppi_c",
    "coa_c",
    "co2_c",
    "amp_c",
    "ACP_c",
    "nh4_c",
    "pi_p",
    "o2_c",
    "fe2_c",
    "h2o2_c",
    "na1_p"
]
util.save("cofactors",cofactors)

#Cofactor pair analysis
pair_count = {}
for rxn in util.model.reactions:
    for met in rxn.metabolites: 
        #Counting all reactant-product pairs not involving cofactors
        if met.id not in cofactors:
            if met not in pair_count:
                pair_count[met] = {}
            if rxn.metabolites[met] < 0:
                for pmet in rxn.metabolites:
                    if pmet.id not in cofactors:
                        if pmet not in pair_count:
                            pair_count[pmet] = {}
                        if rxn.metabolites[pmet] > 0:
                            if pmet not in pair_count[met]:
                                pair_count[met][pmet] = 0
                            pair_count[met][pmet] += 1
                            if met not in pair_count[pmet]:
                                pair_count[pmet][met] = 0
                            pair_count[pmet][met] += 1
data = []
for met in pair_count:
    for pmet in pair_count[met]:
        if pair_count[met][pmet] > 20:
            data.append({"one":met.id,"two":pmet.id,"count":pair_count[met][pmet]})
df = pd.DataFrame.from_records(data)
df.to_csv(util.output_dir+"/PairCounts.csv")
pairs = [
["nadph_c","nadp_c"],
["nadp_c","nadph_c"],
#["pyr_c","pep_c"],
["nad_c","nadh_c"],
["nadh_c","nad_c"],
#["pep_c","pyr_c"],
#["mqn8_c","mql8_c"],
#["mql8_c","mqn8_c"],
#["q8_c","q8h2_c"],
#["q8h2_c","q8_c"],
["fad_c","fadh2_c"],
["fadh2_c","fad_c"]
]
util.save("pairs",pairs)

rxn_cof = {}
for rxn in util.model.reactions:
    rxn_cof[rxn.id] = {}
    for met in rxn.metabolites:
        if rxn.metabolites[met] < 0:
            for pair in pairs:
                if met.id == pair[0]:
                    for pmet in rxn.metabolites:
                        if rxn.metabolites[pmet] > 0:
                            if pmet.id == pair[1]:
                                rxn_cof[rxn.id][met] = 1
                                rxn_cof[rxn.id][pmet] = 1
util.save("rxn_cof",rxn_cof)

Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/PLP/nboutput when using KBDevUtils.output_dir


1722632118.271793 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Classifying reactions - V1.0

In [12]:
%run plputil.py
pairings = util.load("pairings")
reaction_expression = util.load("reaction_expression")
fluxes = util.load("fluxes")
flux_fold_changes = util.load("flux_fold_changes")
expression_fold_changes = util.load("expression_fold_changes")
rxn_cof = util.load("rxn_cof")
met_class = util.load("met_class")

for item in pairings:
    classes = {}
    class_data = {}
    class_suffix = {}
    rxncls = {}
    prodcls = {}
    rxn_class = {}
    for rxn in util.model.reactions:
        if abs(fluxes[pairings[item][0]][rxn.id]) + abs(fluxes[pairings[item][0]][rxn.id]) > 0.0000001:
            flux = fluxes[pairings[item][0]][rxn.id]
            if abs(flux) < 0.0000001:
                flux = fluxes[pairings[item][1]][rxn.id]
            fluxcls = "N"
            expcls="N"
            if rxn.id in flux_fold_changes[item]:
                if flux_fold_changes[item][rxn.id] > 0.1:
                    fluxcls = "U"
                elif flux_fold_changes[item][rxn.id] < -0.1:
                    fluxcls = "D"
                else:
                    fluxcls = "S"
            if rxn.id in expression_fold_changes[item]:
                if expression_fold_changes[item][rxn.id] > 0.1:
                    expcls = "U"
                elif expression_fold_changes[item][rxn.id] < -0.1:
                    expcls = "D"
                else:
                    expcls = "S"
            rxn_class[rxn.id] = fluxcls+expcls
            for met in rxn.metabolites:
                if met.id not in cofactors and met not in rxn_cof[rxn.id]:
                    if met in met_class:
                        if rxn.metabolites[met]*flux < 0:
                            if rxn.id not in rxncls:
                                rxncls[rxn.id] = {}
                            if met_class[met] not in rxncls[rxn.id]:
                                rxncls[rxn.id][met_class[met]] = 0
                            rxncls[rxn.id][met_class[met]] += 1
                        elif rxn.metabolites[met]*flux > 0:
                            if rxn.id not in prodcls:
                                prodcls[rxn.id] = {}
                            if met_class[met] not in prodcls[rxn.id]:
                                prodcls[rxn.id][met_class[met]] = 0
                            prodcls[rxn.id][met_class[met]] += 1
 
    stop = False
    propagated_reactions = {}
    count = 0
    while(not stop):
        new_rxn = []
        for rxn in util.model.reactions:
            if abs(fluxes[pairings[item][0]][rxn.id]) + abs(fluxes[pairings[item][0]][rxn.id]) > 0.0000001:
                flux = fluxes[pairings[item][0]][rxn.id]
                if abs(flux) < 0.0000001:
                    flux = fluxes[pairings[item][1]][rxn.id]
                if rxn.id not in propagated_reactions:
                    if rxn.id in rxncls or rxn.id in prodcls:
                        propagated_reactions[rxn.id] = True
                        for met in rxn.metabolites:
                            if met.id not in cofactors and met not in rxn_cof[rxn.id]:
                                for neighbor_rxn in met.reactions:
                                    if met not in rxn_cof[neighbor_rxn.id]:
                                        if abs(fluxes[pairings[item][0]][neighbor_rxn.id]) + abs(fluxes[pairings[item][0]][neighbor_rxn.id]) > 0.0000001:
                                            nflux =fluxes[pairings[item][0]][rxn.id]
                                            if abs(nflux) < 0.0000001:
                                                nflux = fluxes[pairings[item][1]][rxn.id]
                                            if neighbor_rxn.id not in propagated_reactions:
                                                if flux*nflux > 0:
                                                    #Same direction
                                                    if rxn.metabolites[met] > 0:
                                                        if neighbor_rxn.metabolites[met] > 0:
                                                            #Both reactions produce/consume a common product/reactant
                                                            pass
                                                        else:
                                                            #Rxn -> neighbor
                                                            sign = 1
                                                            if flux < 0:
                                                                #Neighbor -> Rxn
                                                                sign = -1
                                                            new_rxn.append(neighbor_rxn.id)
                                                            util.process_reaction_class(sign,rxncls,rxn.id,neighbor_rxn.id)
                                                            util.process_reaction_class(sign,prodcls,rxn.id,neighbor_rxn.id)
                                                    else:
                                                        if neighbor_rxn.metabolites[met] > 0:
                                                            #Neighbor -> Rxn 
                                                            sign = -1
                                                            if flux < 0:
                                                                #Neighbor -> Rxn
                                                                sign = 1
                                                            new_rxn.append(neighbor_rxn.id)
                                                            util.process_reaction_class(sign,rxncls,rxn.id,neighbor_rxn.id)
                                                            util.process_reaction_class(sign,prodcls,rxn.id,neighbor_rxn.id)
                                                        else:
                                                            #Both reactions produce/consume a common product/reactant
                                                            pass
                                                else:
                                                    #Opposite direction
                                                    if rxn.metabolites[met] > 0:
                                                        if neighbor_rxn.metabolites[met] > 0:
                                                            #Rxn -> neighbor
                                                            sign = 1
                                                            if flux < 0:
                                                                #Neighbor -> Rxn
                                                                sign = -1
                                                            new_rxn.append(neighbor_rxn.id)
                                                            util.process_reaction_class(sign,rxncls,rxn.id,neighbor_rxn.id)
                                                            util.process_reaction_class(sign,prodcls,rxn.id,neighbor_rxn.id)
                                                        else:
                                                            pass
                                                    else:
                                                        if neighbor_rxn.metabolites[met] > 0:
                                                            pass
                                                        else:
                                                            #Neighbor -> Rxn
                                                            sign = -1
                                                            if flux < 0:
                                                                #Rxn -> neighbor
                                                                sign = 1
                                                            new_rxn.append(neighbor_rxn.id)
                                                            util.process_reaction_class(sign,rxncls,rxn.id,neighbor_rxn.id)
                                                            util.process_reaction_class(sign,prodcls,rxn.id,neighbor_rxn.id)
        if len(new_rxn) == 0:
            stop = True
    rxndata = []
    classdata = []
    atomicreg = {}
    rxnpathway = {}
    classmeta = {}
    for rxnid in rxn_class:
        atomicreg[rxnid] = None
        rxnpathway[rxnid] = None
        react = ""
        prod = ""
        dcount = 0
        ucount = 0
        scount = 0
        reactstr = ""
        prodstr = ""
        if rxnid in rxncls:
            for currcls in rxncls[rxnid]:
                if len(reactstr) > 0:
                    reactstr += ";"
                reactstr += currcls + str(rxncls[rxnid][currcls])
                basecls = currcls[-1:]
                newval = 0
                if len(currcls) > 1:
                    newval = int(currcls[0:-1])
                if basecls == "U":
                    ucount += rxncls[rxnid][currcls]/(1+abs(newval))
                elif basecls == "D":
                    dcount += rxncls[rxnid][currcls]/(1+abs(newval))
                else:
                    scount += rxncls[rxnid][currcls]/(1+abs(newval))
        if dcount+ucount+scount > 0:
            react = "S"
            if dcount > ucount+1:
                react = "D"
            elif dcount > ucount:
                react = "D"
            elif dcount < ucount+1:
                react = "U"
            elif dcount > ucount:
                react = "U"
        dcount = 0
        ucount = 0
        scount = 0
        if rxnid in prodcls:
            for currcls in prodcls[rxnid]:
                if len(prodstr) > 0:
                    prodstr += ";"
                prodstr += currcls + str(prodcls[rxnid][currcls])
                basecls = currcls[-1:]
                newval = 0
                if len(currcls) > 1:
                    newval = int(currcls[0:-1])
                if basecls == "U":
                    ucount += prodcls[rxnid][currcls]/(1+abs(newval))
                elif basecls == "D":
                    dcount += prodcls[rxnid][currcls]/(1+abs(newval))
                else:
                    scount += prodcls[rxnid][currcls]/(1+abs(newval))
        if dcount+ucount+scount > 0:
            prod = "S"
            if dcount > ucount+1:
                prod = "D"
            elif dcount > ucount:
                prod = "D"
            elif dcount < ucount+1:
                prod = "U"
            elif dcount > ucount:
                prod = "U"
        rxn = util.model.reactions.get_by_id(rxnid)
        superclass = react+";"+prod+";"+rxn_class[rxnid]
        rxndata.append({"ID":rxnid,"Equation":rxn.build_reaction_string(),"Name":rxn.name,"Superclass":superclass,"Base class":rxn_class[rxnid],"React":react,"Prod":prod,"ReactStr":reactstr,"ProdStr":prodstr,"AtomicRegulon":atomicreg[rxnid],"Pathway":rxnpathway[rxnid]})
        if superclass not in classmeta:
            classes[superclass] = []
            classmeta[superclass] = {"base":rxn_class[rxnid],"react":react,"prod":prod}
        classes[superclass].append(rxnid)
    classdata=[]
    for superclass in classes:
        classdata.append({"Class":superclass,"Count":len(classes[superclass]),"Reactions":",".join(classes[superclass]),"Base":classmeta[superclass]["base"],"Prod":classmeta[superclass]["prod"],"React":classmeta[superclass]["react"]})
    df = pd.DataFrame.from_records(classdata)
    df.to_csv(util.output_dir+"/"+item+"ReactionClasses.csv")
    df = pd.DataFrame.from_records(rxndata)
    df.to_csv(util.output_dir+"/"+item+"ReactionList.csv")

Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/PLP/nboutput when using KBDevUtils.output_dir


1722659470.822721 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Classifying Reactions V2.0

In [15]:
%run plputil.py
pairings = util.load("pairings")
reaction_expression = util.load("reaction_expression")
fluxes = util.load("fluxes")
flux_fold_changes = util.load("flux_fold_changes")
expression_fold_changes = util.load("expression_fold_changes")
rxn_cof = util.load("rxn_cof")
met_class = util.load("met_class")

for item in pairings:
    rxn_flux = {}
    rxn_class = {}
    for rxn in util.model.reactions:
       if abs(fluxes[pairings[item][0]][rxn.id]) + abs(fluxes[pairings[item][0]][rxn.id]) > 0.0000001:
            flux = fluxes[pairings[item][0]][rxn.id]
            if abs(flux) < 0.0000001:
                flux = fluxes[pairings[item][1]][rxn.id]
            fluxcls = "N"
            expcls="N"
            if rxn.id in flux_fold_changes[item]:
                if flux_fold_changes[item][rxn.id] > 0.1:
                    fluxcls = "U"
                elif flux_fold_changes[item][rxn.id] < -0.1:
                    fluxcls = "D"
                else:
                    fluxcls = "S"
            if rxn.id in flux_fold_changes[item]:
                if flux_fold_changes[item][rxn.id] > 0.1:
                    expcls = "U"
                elif flux_fold_changes[item][rxn.id] < -0.1:
                    expcls = "D"
                else:
                    expcls = "S"
            rxn_flux[rxn.id] = flux
            rxn_class[rxn.id] = fluxcls+expcls
    asserted_met_class = {}
    asserted_met_score = {}
    cycles = 1
    rxn_react = {}
    rxn_prod = {}
    stop = False
    while(stop == False):
        stop = True
        rxn_react = {}
        rxn_prod = {}
        for rxn in util.model.reactions:
            if rxn.id in rxn_flux:
                rdc = ruc = pdc = puc = rsc = psc = 0
                for met in rxn.metabolites:
                    if met.id not in cofactors and met not in rxn_cof[rxn.id]:
                        if rxn.metabolites[met]*flux < 0:
                            if rxn.id not in rxn_react:
                                rxn_react[rxn.id] = {"U":0,"D":0,"S":0}
                            if met in met_class:
                                if met_class[met] == "D":
                                    rxn_react[rxn.id]["D"] += 1    
                                elif met_class[met] == "U":
                                    rxn_react[rxn.id]["U"] += 1
                                else:
                                    rxn_react[rxn.id]["S"] += 1
                            elif met in asserted_met_class:
                                if asserted_met_class[met] == "D":
                                    rxn_react[rxn.id]["D"] += asserted_met_score[met]    
                                elif asserted_met_class[met] == "U":
                                    rxn_react[rxn.id]["U"] += asserted_met_score[met]
                                else:
                                    rxn_react[rxn.id]["S"] += asserted_met_score[met]
                        elif rxn.metabolites[met]*flux > 0:
                            if rxn.id not in rxn_prod:
                                rxn_prod[rxn.id] = {"U":0,"D":0,"S":0}
                            if met in met_class:
                                if met_class[met] == "D":
                                    rxn_prod[rxn.id]["D"] += 1
                                elif met_class[met] == "U":
                                    rxn_prod[rxn.id]["U"] += 1
                                else:
                                    rxn_prod[rxn.id]["S"] += 1
                            elif met in asserted_met_class:
                                if asserted_met_class[met] == "D":
                                    rxn_prod[rxn.id]["D"] += asserted_met_score[met]    
                                elif asserted_met_class[met] == "U":
                                    rxn_prod[rxn.id]["U"] += asserted_met_score[met]
                                else:
                                    rxn_prod[rxn.id]["S"] += asserted_met_score[met]    
        for met in util.model.metabolites:
            if met not in met_class and met not in asserted_met_class: 
                uc = dc = sc = 0
                data = 0
                nodata = 0
                for rxn in met.reactions:
                    if met.id not in cofactors and met not in rxn_cof[rxn.id]:
                        if rxn.metabolites[met]*flux < 0:
                            if rxn.id in rxn_prod:
                                uc += rxn_prod[rxn.id]["U"]
                                dc += rxn_prod[rxn.id]["D"]
                                sc += rxn_prod[rxn.id]["S"]
                                data += 1
                            else:
                                nodata += 1
                        elif rxn.metabolites[met]*flux > 0:
                            if rxn.id in rxn_react:
                                uc += rxn_react[rxn.id]["U"]
                                dc += rxn_react[rxn.id]["D"]
                                sc += rxn_react[rxn.id]["S"]
                                data += 1
                            else:
                                nodata += 1
                if uc + dc + sc > 0 and data/(data+nodata) >= 0.5:
                    stop = False
                    if uc > dc:
                        asserted_met_class[met] = "U"
                    elif dc > uc:
                        asserted_met_class[met] = "D"
                    else:
                        asserted_met_class[met] = "S"
                    asserted_met_score[met] = 1/(1+cycles)
        cycles += 1
    rxn_react = {}
    rxn_prod = {}
    rxn_react_str = {}
    rxn_prod_str = {}
    for rxn in util.model.reactions:
        if rxn.id in rxn_flux:
            rdc = ruc = pdc = puc = rsc = psc = 0
            for met in rxn.metabolites:
                if met.id not in cofactors and met not in rxn_cof[rxn.id]:
                    if rxn.metabolites[met]*flux < 0:
                        if rxn.id not in rxn_react:
                            rxn_react[rxn.id] = {"U":0,"D":0,"S":0}
                            rxn_react_str[rxn.id] = ""
                        if met in met_class:
                            rxn_react_str[rxn.id] += met_class[met]+"1"
                            if met_class[met] == "D":
                                rxn_react[rxn.id]["D"] += 1
                            elif met_class[met] == "U":
                                rxn_react[rxn.id]["U"] += 1
                            else:
                                rxn_react[rxn.id]["S"] += 1
                        elif met in asserted_met_class:
                            rxn_react_str[rxn.id] += asserted_met_class[met]+str(asserted_met_score[met])
                            if asserted_met_class[met] == "D":
                                rxn_react[rxn.id]["D"] += asserted_met_score[met]    
                            elif asserted_met_class[met] == "U":
                                rxn_react[rxn.id]["U"] += asserted_met_score[met]
                            else:
                                rxn_react[rxn.id]["S"] += asserted_met_score[met]
                    elif rxn.metabolites[met]*flux > 0:
                        if rxn.id not in rxn_prod:
                            rxn_prod[rxn.id] = {"U":0,"D":0,"S":0}
                            rxn_prod_str[rxn.id] = ""
                        if met in met_class:
                            rxn_prod_str[rxn.id] += met_class[met]+"1"
                            if met_class[met] == "D":
                                rxn_prod[rxn.id]["D"] += 1
                            elif met_class[met] == "U":
                                rxn_prod[rxn.id]["U"] += 1
                            else:
                                rxn_prod[rxn.id]["S"] += 1
                        elif met in asserted_met_class:
                            rxn_prod_str[rxn.id] += asserted_met_class[met]+str(asserted_met_score[met])
                            if asserted_met_class[met] == "D":
                                rxn_prod[rxn.id]["D"] += asserted_met_score[met]    
                            elif asserted_met_class[met] == "U":
                                rxn_prod[rxn.id]["U"] += asserted_met_score[met]
                            else:
                                rxn_prod[rxn.id]["S"] += asserted_met_score[met]    
    rxndata = []
    classes = {}
    classdata = []
    atomicreg = {}
    rxnpathway = {}
    classmeta = {}
    for rxnid in rxn_class:
        atomicreg[rxnid] = None
        rxnpathway[rxnid] = None
        react = ""
        if rxnid in rxn_react:
            if rxn_react[rxnid]["U"] > rxn_react[rxnid]["D"]:
                react = "U"
            elif rxn_react[rxnid]["U"] < rxn_react[rxnid]["D"]:
                react = "D"
            else:
                react = "S"
        prod = ""
        if rxnid in rxn_prod:
            if rxn_prod[rxnid]["U"] > rxn_prod[rxnid]["D"]:
                prod = "U"
            elif rxn_prod[rxnid]["U"] < rxn_prod[rxnid]["D"]:
                prod = "D"
            else:
                prod = "S"
        reactstr = ""
        prodstr = ""
        rxn = util.model.reactions.get_by_id(rxnid)
        superclass = react+";"+prod+";"+rxn_class[rxnid]
        rxndata.append({"ID":rxnid,"Equation":rxn.build_reaction_string(),"Name":rxn.name,"Superclass":superclass,"Base class":rxn_class[rxnid],"React":react,"Prod":prod,"ReactStr":reactstr,"ProdStr":prodstr,"AtomicRegulon":atomicreg[rxnid],"Pathway":rxnpathway[rxnid]})
        if superclass not in classmeta:
            classes[superclass] = []
            classmeta[superclass] = {"base":rxn_class[rxnid],"react":react,"prod":prod}
        classes[superclass].append(rxnid)
    classdata=[]
    for superclass in classes:
        classdata.append({"Class":superclass,"Count":len(classes[superclass]),"Reactions":",".join(classes[superclass]),"Base":classmeta[superclass]["base"],"Prod":classmeta[superclass]["prod"],"React":classmeta[superclass]["react"]})
    df = pd.DataFrame.from_records(classdata)
    df.to_csv(util.output_dir+"/"+item+"ReactionClasses2.csv")
    df = pd.DataFrame.from_records(rxndata)
    df.to_csv(util.output_dir+"/"+item+"ReactionList2.csv")

Output files printed to:/Users/chenry/code/notebooks/ModelSEED2Notebooks/PLP/nboutput when using KBDevUtils.output_dir


1722660885.486455 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


In [27]:
table = pd.DataFrame.from_records([{"Object":"Test"}])
for index, row in table.iterrows():
    table.at[index,'Object'] = '<a href="javascript:view_annotations('+"'"+row["Object"]+"'"+')">'+row["Object"]+"</a>"
json_str = '{"data":'+table.to_json(orient='records')+'}'
with open("test.json", 'w') as f:
    f.write(json_str)

In [ ]:
#Model WT elevated PLP vs yGGS KO - keep flux the same between both conditions; compute Kapp; compare kapp with metabolite data
